<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# **<font color="#76b900">노트북 4: 시퀀스 변환을 위한 인코더 모델-디코더 모델</font>**

이전 노트북에서는 인코더 전용 모델이 **정적인 입력**(예: 분류, 감정 분석, 개체명 인식 등)을 처리할 때 뛰어난 성능을 보임을 확인했습니다.  
하지만 실제 자연어 작업(번역, 요약, 질의응답 등)은 입력을 받아서 **"동적으로" 출력 시퀀스를 생성**해야 하므로, 더 복잡한 구조가 필요합니다.

###  학습 목표
- 인코더-디코더 구조의 작동 원리 이해  
- 입력 시퀀스를 받아 새로운 시퀀스를 생성하는 메커니즘 탐색  
- 대표 모델 **T5**의 구조 및 활용 사례 학습  
- 번역/요약 작업에서 인코더-디코더 구조의 장점 이해  



### 주요 학습 내용
- 인코더와 디코더 각각의 역할과 정보 흐름  
- **크로스 어텐션(Cross-Attention)** 메커니즘의 필요성과 작동 방식  
- Transformer 기반 인코더-디코더 아키텍처 개요  
- T5 모델을 활용한 번역 및 요약 실습 (HuggingFace Pipeline 사용)

## **Part 4.1: 기계 번역 작업**

**시퀀스 변환 모델이란?**
시퀀스 변환 모델은 입력 시퀀스를 받아 출력 시퀀스로 변환하는 모델입니다.  
대표적인 예로는 번역(예: 영어 → 프랑스어), 요약, 질의응답 등이 있습니다.


[**기계 번역**](https://huggingface.co/tasks/translation)은 한 언어의 텍스트를 다른 언어로 자동 변환하는 작업입니다.

겉보기엔 쉬워 보여도,언어별로 **문법**·**어순**·**문화적 뉘앙스**가 모두 다르고  
문장을 통째로 이해해서 자연스럽게 변환해야 하므로 실제로는 매우 어려운 AI 과제입니다.


### **인코더와 디코더의 협업**

- **인코더**: 입력 언어의 의미를 벡터로 압축하여 표현  
- **디코더**: 그 벡터를 바탕으로 목표 언어 문장을 하나씩 생성  

이 구조 덕분에 **하나의 셀이 두 언어를 모두 완벽히 이해할 필요가 없습니다.**  
인코더는 입력 언어의 문맥만 잘 파악하고, 디코더는 목표 언어 생성 규칙에 집중할 수 있어 학습 부담이 줄어듭니다.


### 인코더-디코더 구조 개요

<img src='imgs/encoder-decoder.png' width=500>

### 예시로 구조 다시 보기

- **T5/BART 구조**:  
  입력 문장 → 인코더 → 의미 벡터 → 디코더 (+크로스 어텐션) → 번역/요약 결과 생성

디코더는 이전에 생성한 단어들을 기반으로 한 단어씩 생성합니다.

### 크로스 어텐션(Cross-Attention)이란?

> **디코더가 출력을 생성할 때, 인코더에서 출력된 의미 벡터에 주의를 집중하는 메커니즘입니다.**

- **Self-Attention**은 디코더가 이전 단어들 간 상호작용을 보는 것  
- **Cross-Attention**은 디코더가 인코더의 모든 출력을 참고하면서 새로운 단어를 생성함  

입력과 출력이 명확히 분리된 구조에서 **크로스 어텐션은 핵심적인 연결 고리 역할**을 수행합니다.
디코더는 **입력 문장의 어떤 단어가 중요한지 선택**하며 출력을 생성합니다.

 예시:  
입력 문장: "I love spicy food."
디코더가 "맵다" vs "짜다" 중 어떤 단어를 생성할지 고민하는 상황일 때,  
인코더에서 "spicy"라는 단어에 집중하여 → "맵다"를 선택

## **Part 4.2: T5 모델을 통한 번역 실습**

In [ ]:
# transformers 라이브러리에서 pipeline 모듈을 임포트합니다.
# pipeline은 복잡한 설정 없이도 다양한 자연어처리 작업을 손쉽게 수행할 수 있게 해주는 기능입니다.
from transformers import pipeline

# 영어를 프랑스어로 번역하는 파이프라인을 생성합니다.
translator = pipeline('translation_en_to_fr', model='t5-base', device="cuda")

# 여러 문장을 한번에 번역합니다. 리스트 형태로 입력하면 각 문장이 순차적으로 번역됩니다.
translator(["Hello World! How's it going?", "What's your name?"])

### T5의 instruction 기반 구조

T5는 단순히 입력만 넣는 것이 아니라, 다음과 같은 **문맥 지시어(Instructions)**를 함께 전달합니다:

```text
translate English to French: Hello World!
```

이런 입력 구조를 통해 하나의 모델로 다양한 작업(번역, 요약, QA 등)을 처리할 수 있습니다.


### T5 번역 구조 시각화

```
 ┌─────────────────────┐
 │   입력 텍스트 (EN)   │
 │ "Hello World!" 등   │
 └─────────┬───────────┘
           │
           ▼
 ┌────────────────────────────┐
 │      전처리 (Preprocess)     │
 │ - "translate English to     │
 │   French: Hello World!"     │
 │ - 토크나이징(tokenization) │
 └─────────┬──────────────────┘
           │
           ▼
 ┌────────────────────────────┐
 │        인코더 (Encoder)      │
 │ - 입력 시퀀스 → 의미 벡터     │
 └─────────┬──────────────────┘
           │
           ▼
 ┌────────────────────────────┐
 │        디코더 (Decoder)      │
 │ - 의미 벡터 기반 토큰 생성    │
 │ - 한 토큰씩 반복 예측         │
 └─────────┬──────────────────┘
           │
           ▼
 ┌────────────────────────────┐
 │      후처리 (Postprocess)    │
 │ - 토큰 시퀀스 → 문장 생성     │
 │ - "Bonjour le monde !" 등   │
 └────────────────────────────┘
```

---

### 각 단계 요약

| 단계 | 설명 |
|------|------|
| **입력** | 영어 원문 또는 태스크 지시어 포함 문장 |
| **전처리** | 입력을 토크나이즈, T5 포맷에 맞춰 인코딩 |
| **인코더** | 입력 시퀀스를 의미 벡터로 변환 |
| **디코더** | 인코더 출력 벡터 기반으로 프랑스어 토큰 생성 |
| **후처리** | 생성된 토큰 시퀀스를 사람이 읽을 수 있는 문장으로 디코딩 |


#### 참고: 코드에서 인코더-디코더 구조 확인

이 아키텍처를 더 자세히 살펴보기 위해 모델 구조를 출력해볼 수 있습니다.  
다만 출력 내용이 매우 복잡하기 때문에, 코드 셀 아래에서 정리된 이미지를 확인하는 것을 권장드립니다

In [ ]:
translator.model           # 전체 모델 구조(매우 복잡)
translator.model.encoder   # 인코더(입력→의미 벡터) 구조(BERT와 유사)
translator.model.decoder   # 디코더(의미 벡터→문장 생성) 구조

<div><img src="imgs/t5-architecture.png" 
     alt="Encoder-Decoder Architecture"
     width="1200"/></div>

T5 모델은 다음과 같은 방식으로 추론을 수행합니다:

> `인코더("{P}: {Q}")`는 `디코더("<pad>")`의 생성을 조건화하며,  
> 디코더는 종료 토큰에 도달할 때까지 한 번에 하나의 토큰을 생성하여 최종 답변 $A$를 만들어냅니다.

이러한 형식은 사전 학습 단계에서 대규모 텍스트 데이터에 적용되며,  
모델이 이 구조를 학습하도록 유도합니다.

> <div><img src="imgs/t5-pic.jpg" width="800"/></div>
>
> **출처: [텍스트-투-텍스트 트랜스포머를 이용한 전이 학습의 한계 탐구](https://arxiv.org/abs/1910.10683v4)**

## **Part 4.3: 범용 인코더-디코더와 토큰 스트리밍 실습**

FLAN-T5처럼 다양한 태스크에 대응하는 인코더-디코더 모델은 여러 자연어 작업(번역, 요약, QA 등)에 두루 쓰입니다.  

이번 파트에서는 FLAN-T5로 영어 → 프랑스어 번역 결과를 **토큰 단위로 실시간 스트리밍** 형태로 출력해보며, 생성 과정을 이해할 수 있도록 합니다.

In [ ]:
import torch

def get_token_generator(pipeline, model=None, tokenizer=None, max_tokens=50):
    # 토큰을 순차적으로 생성하는 제너레이터를 초기화하는 함수
    # pipeline: 번역이나 생성을 수행할 모델 파이프라인
    # max_tokens: 생성할 최대 토큰 수 (기본값 50)
    
    model = pipeline.model or model
    tknzr = pipeline.tokenizer or tokenizer
    encoder = getattr(model, "encoder", None)
    # 디코더 컴포넌트 찾기 (여러 모델 구조에서 작동하도록 다양한 속성명 시도)~
    decoder = (
        getattr(model, "decoder", None) 
        or getattr(model, "model", None) 
        or getattr(model, "transformer", None)
    )
    lm_head = getattr(model, "lm_head", None)  # 언어 모델의 출력층
    dev = decoder.device  # 모델이 실행되는 디바이스 (CPU/GPU)
    
    def token_generator(
        encoder_input: str = "",
        decoder_input: str = "",
        max_tokens: int = max_tokens
    ):
        # 입력 텍스트를 토큰 인덱스로 변환 (마지막 토큰 제외)
        encoder_input_idxs = tknzr.encode(encoder_input)[:-1] * bool(encoder_input)
        decoder_input_idxs = tknzr.encode(decoder_input)[:-1] * bool(decoder_input)
        decoder_inputs = {}

        # [인코더-디코더] 입력 컨텍스트를 디코더의 조건부 은닉 상태로 변환
        if encoder:
            encoder_inputs = {"input_ids": torch.tensor([encoder_input_idxs], device=dev)}
            encoder_outputs = encoder(**encoder_inputs)
            decoder_inputs["encoder_hidden_states"] = encoder_outputs.last_hidden_state
        elif encoder_input_idxs:
            print("인코더가 없는데 encoder_input이 지정되었습니다. 무시합니다.")
            
        # [인코더-디코더/디코더] <pad>부터 시작하여 </s>(종료 토큰)가 나올 때까지 디코딩
        buffer_token_idxs = [] if (tknzr.pad_token_id is None) else [tknzr.pad_token_id]
        buffer_token_idxs += decoder_input_idxs
        buffer_token_str = ""
        max_length = len(buffer_token_idxs) + max_tokens
        
        while len(buffer_token_idxs) < max_length:
            # 현재 버퍼를 디코더에 입력하고, 인코더 상태도 함께 전달
            # 참고: 여기서는 마지막 은닉 상태만 사용하지만, 일부 모델은 더 많은 상태를 사용
            
            decoder_inputs["input_ids"] = torch.tensor([buffer_token_idxs], device=dev).long()
            decoder_outputs = decoder(**decoder_inputs)
            model_outputs = lm_head(decoder_outputs.last_hidden_state)
        
            # 가장 확률이 높은 다음 토큰을 선택하고 버퍼에 추가
            try: sampled_token_idx = torch.argmax(model_outputs, -1)[0][-1].item()
            except: break
            buffer_token_idxs += [sampled_token_idx]
            buffer_token_old = buffer_token_str
            buffer_token_str = tknzr.decode(buffer_token_idxs)
            buffer_token_new = buffer_token_str[len(buffer_token_old):]

            # 새로운 토큰을 생성하고 yield로 반환
            # 만약 종료 토큰(</s>)이 나오면 생성 중단
            if sampled_token_idx == tknzr.eos_token_id:
                break
            if buffer_token_new:
                yield buffer_token_new
    
    return token_generator

###############################################################################

# 번역기 파이프라인으로부터 토큰 생성기 초기화
streamer = get_token_generator(translator)
# 영어에서 프랑스어로 번역할 입력 문장
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

# 토큰을 하나씩 생성하며 출력 (|로 구분)
for token in streamer(encoder_input = input_raw_str):
    print(token, end="|")

In [ ]:
# Google의 FLAN-T5 Large 모델을 사용하여 텍스트 생성 파이프라인을 설정합니다.
# FLAN-T5는 다양한 자연어 처리 작업(번역, 요약, 질문답변 등)을 수행할 수 있는 강력한 언어 모델입니다.
flan_t5_pipe = pipeline("text2text-generation", model="google/flan-t5-large")

# 토큰 생성기를 설정합니다.
# 토큰 생성기는 모델의 출력을 토큰 단위로 실시간 스트리밍하는데 사용됩니다.
# 이를 통해 긴 텍스트 생성 시 결과를 점진적으로 확인할 수 있습니다.
streamer = get_token_generator(flan_t5_pipe)

# 번역할 영어 문장을 입력합니다.
# </s>는 입력 텍스트의 끝을 나타내는 특수 토큰입니다.
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

# 토큰 생성기를 사용하여 번역을 수행하고, 결과를 토큰 단위로 출력합니다.
# end=""를 사용하여 토큰들이 연속적으로 출력되도록 합니다.
for token in streamer(encoder_input = input_raw_str):
    print(token, end="")
'''

이 코드는 FLAN-T5 모델을 사용하여 영어 문장을 프랑스어로 번역하는 예제입니다. 특징적인 점은 번역 결과를 한 번에 출력하지 않고, 토큰 단위로 실시간 스트리밍한다는 것입니다. 이러한 방식은 특히 긴 텍스트를 처리할 때 사용자에게 중간 결과를 보여줄 수 있어 유용합니다.

FLAN-T5는 Google에서 개발한 범용 언어 모델로, 다양한 자연어 처리 작업을 수행할 수 있습니다. "Large" 버전은 모델의 크기를 나타내며, 더 정확한 결과를 제공할 수 있지만 더 많은 컴퓨팅 리소스를 필요로 합니다.
'''

## **Part 4.5: 제로샷 및 멀티태스크 활용**

FLAN-T5와 같은 인코더-디코더 계열의 모델은 다양한 자연어 처리 작업을 **지시문 기반(Text-to-Text)** 형식으로 수행할 수 있습니다.

이러한 구조는 하나의 모델로 다양한 작업을 동시에 수행하는 **멀티태스크(Multi-Task)** 설정을 자연스럽게 지원합니다.

이러한 능력은 FLAN-T5가 다양한 작업과 지시문에 대한 Instruction-Tuning을 통해 **범용적인 언어 이해 능력**을 학습했기 때문에 가능합니다.

아래는 FLAN-T5를 활용하여 다양한 태스크를 지시문만으로 실행하는 예시입니다:

In [ ]:
# FLAN-T5 모델 기반 text-to-text 파이프라인 생성
pipe = pipeline("text2text-generation", model="google/flan-t5-base")

# 감성 분류 (Sentiment Analysis)
print(pipe("classify sentiment: I love this place.")[0]["generated_text"])

# 문법 교정 (Grammar Correction)
print(pipe("Fix grammar: She go to school every day.")[0]["generated_text"])

# 카테고리 분류 (Topic Classification)
print(pipe("Which category does this sentence belong to: I paid with my Visa credit card.")[0]["generated_text"])

이처럼 T5 계열 모델은 입력을 **자연어 지시문(prompt)** 형태로 구성하기만 하면,  
**특정 태스크에 대해 별도의 파인튜닝(fine-tuning) 없이도** 새로운 작업을 제로샷 방식으로 수행할 수 있습니다.

| 개념 | 설명 |
|------|------|
| **멀티태스크** | 분류, 요약, 번역 등 여러 작업을 하나의 모델로 동시에 처리 |
| **제로샷** | 새 입력에 대해 지시문만으로 수행 |

# <font color="#76b900">**마무리**</font>

인코더-디코더 구조는 입력과 출력이 명확하게 분리된 작업에 매우 효과적입니다.

특히 번역, 요약, 질의응답 등에서 많이 활용됩니다.

**T5/BART**는 대표적인 구조이며, 다양한 instruction을 통해 다중 태스크 처리가 가능합니다  

디코더는 **자가회귀 방식**으로 하나씩 토큰을 생성하며, **크로스 어텐션**을 통해 인코더 정보에 기반한 응답을 생성합니다

또한, FLAN-T5와 같은 모델은 다양한 태스크 유형에 대한 사전 지시문 학습을 바탕으로, **별도 파인튜닝 없이도** 새로운 입력을 처리하는 **제로샷 추론**을 지원합니다.  
이는 사용자가 단지 자연어 지시문만으로 분류, 요약, 번역 등의 다양한 작업을 수행할 수 있게 해줍니다.

In [ ]:
## 작업이 끝났을 때 실행해주세요!

import IPython
app = IPython.Application.instance() # 현재 실행 중인 IPython 인스턴스(커널)를 가져옵니다.
# 커널을 안전하게 종료합니다.
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>